In [1]:
"""
NOTE:
Before running this, create a virtual environment running
Python 3.12
""" 
# pytorch install, cpu-only
%pip install torch 
%pip install transformers
%pip install sentence-transformers
%pip install pandas
%pip install chonkie
%pip install pypdf[image]

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.



In [17]:
from transformers import AutoTokenizer
from sentence_transformers import SentenceTransformer
from sentence_transformers.models import StaticEmbedding

import os
import sys
import pandas as pd
import numpy as np
# Add the current directory to the path
sys.path.append(os.getcwd())
from preprocess import prepare_PDF

In [3]:
# define paths
main = r"C:\Users\Steven\Documents\Python\super-search"
data = f"{main}/data/tests"
test_file = f"{data}/32286.pdf"

In [4]:
df = pd.DataFrame.from_dict(prepare_PDF(test_file))
df

,raw_chunk,processed_chunk,file_path
0,NBER WORKING PAPER SERIES\nPROMOTING PUBLIC HE...,PROMOTING PUBLIC HEALTH WITH BLUNT INSTRUMENTS...,C:\Users\Steven\Documents\Python\super-search/...
1,"NBER publications.\n© 2024 by Rahi Abouk, John...","NBER publications. 2024 Rahi Abouk, John S. Ea...",C:\Users\Steven\Documents\Python\super-search/...
2,If mandates increase perceived safety of the ...,If mandates increase perceived safety healthca...,C:\Users\Steven\Documents\Python\super-search/...
3,mandates and are slower to be replaced than wo...,mandates slower to replaced than workers in no...,C:\Users\Steven\Documents\Python\super-search/...
4,edu 1 Introduction\nThe COVID-19 pandemic - li...,edu 1 Introduction COVID-19 pandemic - like ot...,C:\Users\Steven\Documents\Python\super-search/...
...,...,...,...
184,"Exitt Entert\nMandate states, Non-mandate Mand...","Exitt Entert Mandate states, Non-mandate Manda...",C:\Users\Steven\Documents\Python\super-search/...
185,Among HCO 0.0539 0.0492 0.0531 0.0481\n(obs: 7...,"Among HCO 0.0539 0.0492 0.0531 0.0481 (obs 7,0...",C:\Users\Steven\Documents\Python\super-search/...
186,50-59 years 0.22 0.22 0.22 0.22\n60-64 years 0...,50-59 years 0.22 0.22 0.22 0.22 60-64 years 0....,C:\Users\Steven\Documents\Python\super-search/...
187,Less than high school 0.02 0.02 0.02 0.02\nHig...,Less than high school 0.02 0.02 0.02 0.02 High...,C:\Users\Steven\Documents\Python\super-search/...


## To-Do:

### Database management
- We need a system to handle vector database as well as allowing for fast retrieval of the files corresponding to each vector
    - Each file needs a file_id
        - Links to the filepath
    - Each chunk needs a chunk_id
        - Links to the chunk text
            - ~~Importantly, link to the original text, not the processed text used for embedding.~~
                - ~~**I think the solution is to do the sentence chunking on the original text, before processing**~~
    - file_id + chunk_id should uniquely identify a vector in the database
- After taking a user query and encoding it, perform similarity search in the database
    to identify a row, then link to the row's filepath and text
- Then print a hyperlink to the file, and print the original text
- **Problem**: how to return images, given that we first caption them with an LLM and then encode the caption?

### Misc.
- Parallelized PDF processing

### Incorporating Images
- Use PyPDF to extract images from each page of the PDF.
```
for page in reader.pages:
    try:
        for image in page.images:
            extract_image(image)
    except:
        pass
```
- To render the image visually: image.image. Can also show as bytes
- Goal is to pass each image to a multi-modal LLM for summary, which is then fed into the cleaned text.
    - Possibly LLaVA for describing the images.
    - Since captioning the images would take a super long time, this should be an optional step.
        - Ideally would be done after first parsing the text, but then you might have to regenerate the whole vector base

### GUI
- The following should be customizable inputs:
    - Chunk token size ("larger is faster but less accurate") (default: 256)
    - Chunk overlap ("larger gives more context per chunk") (default: chunk_size / 4)
    - Choice of sentence transformer: provide a few options based on speed/accuracy tradeoff.
        - Fastest: static-retrieval-mrl-en-v1
        - Medium: bge-m3
        - Slowest: gte-large-en-v1.5
        - (these are subject to change)
    - Index database save location
    - Similarity matrix (default: cosine)

In [10]:
# Current encoding model implementation: static-retrieval-mrl-en-v1
# https://huggingface.co/sentence-transformers/static-retrieval-mrl-en-v1
# Model defaults to 1024 dense dimensions, but can be truncated to save space/time

truncated_dimensions = 1024

model = SentenceTransformer(
    "sentence-transformers/static-retrieval-mrl-en-v1"
    , device="cpu"
    , truncate_dim=truncated_dimensions
    )


In [38]:
df

,raw_chunk,processed_chunk,file_path,vector
0,NBER WORKING PAPER SERIES\nPROMOTING PUBLIC HE...,PROMOTING PUBLIC HEALTH WITH BLUNT INSTRUMENTS...,C:\Users\Steven\Documents\Python\super-search/...,"[1.1682692, -1.5645295, 1.7560189, 0.28907633,..."
1,"NBER publications.\n© 2024 by Rahi Abouk, John...","NBER publications. 2024 Rahi Abouk, John S. Ea...",C:\Users\Steven\Documents\Python\super-search/...,"[-0.15334079, -0.21552537, 0.25327072, -0.5400..."
2,If mandates increase perceived safety of the ...,If mandates increase perceived safety healthca...,C:\Users\Steven\Documents\Python\super-search/...,"[-0.42630705, 3.0673761, 2.570261, -0.12676458..."
3,mandates and are slower to be replaced than wo...,mandates slower to replaced than workers in no...,C:\Users\Steven\Documents\Python\super-search/...,"[-2.0449557, -1.4228649, 0.19119436, -1.424515..."
4,edu 1 Introduction\nThe COVID-19 pandemic - li...,edu 1 Introduction COVID-19 pandemic - like ot...,C:\Users\Steven\Documents\Python\super-search/...,"[-2.1557646, 0.78150314, 0.5351259, -0.6953711..."
...,...,...,...,...
184,"Exitt Entert\nMandate states, Non-mandate Mand...","Exitt Entert Mandate states, Non-mandate Manda...",C:\Users\Steven\Documents\Python\super-search/...,"[1.1938934, 0.5603892, 0.61683416, -0.25354582..."
185,Among HCO 0.0539 0.0492 0.0531 0.0481\n(obs: 7...,"Among HCO 0.0539 0.0492 0.0531 0.0481 (obs 7,0...",C:\Users\Steven\Documents\Python\super-search/...,"[0.5000687, 0.20520635, 0.11815526, -0.7151489..."
186,50-59 years 0.22 0.22 0.22 0.22\n60-64 years 0...,50-59 years 0.22 0.22 0.22 0.22 60-64 years 0....,C:\Users\Steven\Documents\Python\super-search/...,"[-0.28278822, -0.20066796, -0.10967996, -2.181..."
187,Less than high school 0.02 0.02 0.02 0.02\nHig...,Less than high school 0.02 0.02 0.02 0.02 High...,C:\Users\Steven\Documents\Python\super-search/...,"[0.043342467, -1.4540507, -0.09367746, -2.4062..."


In [13]:
vecs = model.encode(df['processed_chunk'])
# This returns a np array of shape (n, d), where n is 
#     number of chunks and d is embedding dimensions.

df2['vector'] = [i for i in np.unstack(vecs)]
# Add the embeddings to our dataframe in a single variable,
#     so each cell contains the d-dimensional np vector.

In [33]:
## TESTING
# Importing a lot of PDFs to see how long this takes
papers_repo = r"C:\Users\Steven\Documents\Python\Data\NBER papers"

files = os.listdir(papers_repo)
files.sort(reverse=True)

full_dict = {
    'raw_chunk': []
    , 'processed_chunk': []
    , 'file_path': []
}

for paper in files[0:99]:
    f = f"{papers_repo}/{paper}"
    iter_dict = prepare_PDF(f)
    full_dict['raw_chunk'].extend(iter_dict['raw_chunk'])
    full_dict['processed_chunk'].extend(iter_dict['processed_chunk'])
    full_dict['file_path'].extend(iter_dict['file_path'])

df = pd.DataFrame.from_dict(full_dict)
df
    

C:\Users\Steven\Documents\Python\Data\NBER papers/9999.pdf
C:\Users\Steven\Documents\Python\Data\NBER papers/9998.pdf
C:\Users\Steven\Documents\Python\Data\NBER papers/9997.pdf
C:\Users\Steven\Documents\Python\Data\NBER papers/9996.pdf
C:\Users\Steven\Documents\Python\Data\NBER papers/9995.pdf
C:\Users\Steven\Documents\Python\Data\NBER papers/9994.pdf
C:\Users\Steven\Documents\Python\Data\NBER papers/9993.pdf
C:\Users\Steven\Documents\Python\Data\NBER papers/9992.pdf
C:\Users\Steven\Documents\Python\Data\NBER papers/9991.pdf
C:\Users\Steven\Documents\Python\Data\NBER papers/9990.pdf
C:\Users\Steven\Documents\Python\Data\NBER papers/9989.pdf
C:\Users\Steven\Documents\Python\Data\NBER papers/9988.pdf
C:\Users\Steven\Documents\Python\Data\NBER papers/9987.pdf
C:\Users\Steven\Documents\Python\Data\NBER papers/9986.pdf
C:\Users\Steven\Documents\Python\Data\NBER papers/9985.pdf
C:\Users\Steven\Documents\Python\Data\NBER papers/9984.pdf
C:\Users\Steven\Documents\Python\Data\NBER papers/9983.p